# このノートブックについて

このノートブックはECHONET Liteプロトコルでエアコンから情報を取得したのち、 [Confluent Kafka Python] を利用し、Pythonを用いてApache Kafkaクラスタにデータを書き込むサンプルである。

[Confluent Kafka Python]: https://docs.confluent.io/ja-jp/clients-confluent-kafka-python/1.5.0/overview.html

# 準備

In [1]:
from confluent_kafka import Producer
from confluent_kafka import Consumer
from confluent_kafka import TopicPartition
from confluent_kafka import OFFSET_BEGINNING
from confluent_kafka.admin import AdminClient
from confluent_kafka.admin import NewTopic
import socket
import time
import os
from threading import Thread
import netifaces
import random

ここでは簡単化のため、単一ノード上にKafka Broker、ZooKeeper、Clientを起動することとする。
そのため、実行ノードの最初にホスト名を取得する。

![動作確認のための簡易アーキテクチャ](https://github.com/dobachi/PythonKafkaECHONETLiteExample/blob/main/images/ArchitectureToWriteECHONETLiteDataToKafka.png?raw=true)

In [2]:
hostname = socket.gethostname()
hostname

'el01'

クライアントに渡すコンフィグを生成する。

In [3]:
producer_conf = {'bootstrap.servers': hostname + ":9092",
                 'client.id': hostname}

In [4]:
consumer_conf = {'bootstrap.servers': hostname + ":9092",
                 'group.id': 0,
                'auto.offset.reset': 'earliest'}

# 実験用のトピックを作る

In [5]:
topic_name = 'el_aircon'
num_partition = 1
replication_factor = 1

コンシューマを作成

In [6]:
consumer = Consumer(consumer_conf)

トピックのリストを得るため、メタデータを取得する。

In [7]:
cluster_metadata = consumer.list_topics()
cluster_metadata

ClusterMetadata(UU5KEDIXTJuFCM8nsa7XcQ)

トピックリストを確認

In [8]:
cluster_metadata.topics

{'el_aircon': TopicMetadata(el_aircon, 1 partitions),
 'test_topic': TopicMetadata(test_topic, 1 partitions),
 '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}

`admin` を取得する。

In [9]:
admin = AdminClient(producer_conf)

トピックを作成する処理を定義

In [10]:
def create_topic(admin, topic_name, num_partition, replication_factor):
    new_topic = NewTopic(topic_name, num_partition, replication_factor)
    result = admin.create_topics([new_topic])
    return result

トピックがあれば削除し、改めて作り直す。（今は動作確認のため、トピック削除をコメントアウト→トピックを流用することにする）

In [11]:
#if topic_name in cluster_metadata.topics:
#   admin.delete_topics([topic_name])

result = create_topic(admin, topic_name, num_partition, replication_factor)   

while(not result['el_aircon'].done()):
    time.sleep(3)

result

{'el_aircon': <Future at 0x7fdfda298e90 state=finished raised KafkaException>}

In [12]:
consumer.close()

# ECHONET Liteでエアコンからデータを読み込む

ECHONET Liteのポート番号、マルチキャストグループのアドレスは変化しないと思うが、もしバッファサイズを変更する場合は以下を編集すること。

In [13]:
EL_PORT = 3610
BUFFER_SIZE = 1024
MULTICAST_GROUP='224.0.23.0'

## リクエスト結果の受信機能

ECHONET Liteを利用し、多様な情報を取得できるが、今回は代表的にエアコンから電源状態と設定値の情報を受け取り、Kafkaに書き込むことにする。

ECHONET Liteのメッセージを受信待ちをするためのスレッドを立ち上げるため、このノートブックを動かしているマシン、インスタンスのIPアドレスを確認するためのヘルパー関数を定義する。

In [14]:
def find_local_ip_addr(find_iface_name=None):
    for iface_name in netifaces.interfaces():
        iface_data = netifaces.ifaddresses(iface_name)
        af_inet = iface_data.get(netifaces.AF_INET)
    
        if not af_inet: continue

        ip_addr = af_inet[0]["addr"]
        
        if find_iface_name == None:
            return ip_addr
        elif iface_name == find_iface_name:
            return ip_addr
    return None

受信したECHONET Liteメッセージの紐解くためのヘルパー関数を定義する。

なお、ここで登場するメッセージの規定は [ECHONET Lite規格書] の中の [家庭用エアコン・HEMS コントローラ間 アプリケーション通信 インタフェース仕様書] に記載されている。
例えば、「２．１ECHONET オブジェクト（EOJ）」節に家庭用エアコンであることを示すためのECHONET Liteオブジェクト（EOJ）の定義が記載されている、など。

[ECHONET Lite規格書]: https://echonet.jp/spec_g/
[家庭用エアコン・HEMS コントローラ間 アプリケーション通信 インタフェース仕様書]: https://echonet.jp/wp/wp-content/uploads/pdf/General/Standard/AIF/ac/ac_aif_ver1.10.pdf

In [15]:
def parse_echonet_res(echonet_res_orig):
    echonet_res = [hex(x) for x in echonet_res_orig]
    res_cols = [echonet_res[ 0: 2],  ## echonetであることの宣言
                echonet_res[ 2: 4],  ## ID
                echonet_res[ 4: 7],  ## SEOJ(送信元機器) 0ef001=ノード
                echonet_res[ 7:10],  ## DEOJ(送信先機器) 05ff01=コントローラ
                echonet_res[10:11],  ## 応答code. 71=set 72=get
                echonet_res[11:12],  ## 処理プロパティ数 2
                echonet_res[12:13],  ## EPC. プロパティ名 80 動作状態
                echonet_res[13:14],  ## PDC. 後のbyte数 1
                echonet_res[14:15],  ## EDT 30:ON、31:OFF
                echonet_res[15:16],  ## EPC. プロパティ名 b3 温度設定値
                echonet_res[16:17],  ## PDC. 後のbyte数 1
                echonet_res[17:18],  ## EDT 温度設定の値
                ]
    return res_cols

受信したECHONET Liteのメッセージから生成されたキー・バリュー形式のデータをApache Kafkaクラスタに書き込むヘルパー関数を定義する。

In [16]:
def produce_to_kafka(producer, key, value):
    def acked(err, msg):
        if err is not None:
            print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
        else:
            print("Message produced: %s" % (str(msg)))

    producer.produce(topic_name, key=key, value=value, callback=acked)

    # Wait up to 1 second for events. Callbacks will be invoked during
    # this method call if the message is acknowledged.
    producer.poll(1)

ECHONET Liteのメッセージを受信するためのスレッドで用いるメソッドを定義する。
なお、ECHONET Liteの受信には基本的なソケットプログラミングの知識が利用できる。

なお、この動作確認では、Apache Kafkaに書き込む際にデバイスIDを付与することにするが、その値はランダムとする。
本当はあらかじめ決められた識別可能なIDと突合して用いることが望ましい。
（今回は簡易的な動作確認目的のため簡略化）

In [17]:
def receive_state():
    # 自身のipアドレスを探索
    local_ip = find_local_ip_addr()
    if local_ip == None:
        return None

    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind(('', EL_PORT))
    # マルチキャストとユニキャストの両方を待ち受け
    sock.setsockopt(socket.IPPROTO_IP,
                    socket.IP_ADD_MEMBERSHIP,
                    socket.inet_aton(MULTICAST_GROUP) + socket.inet_aton(local_ip))

    producer = Producer(producer_conf)
    
    # ここではスレッド起動時にランダムなIDを付与することにする。読みやすさのためゼロパディングしておく。
    ID_MIN = 0
    ID_MAX = 1000
    id = str(random.randint(ID_MIN, ID_MAX)).zfill(4)
    
    while True:
      data = sock.recvfrom(BUFFER_SIZE)
      echonet_res = parse_echonet_res(data[0])
      print('# original')
      print(data)
      print('# parsed')
      print(echonet_res)
      key = '%s#' % (id) + ','.join(echonet_res[1])
      value = "%s,%s" % (int(echonet_res[8][0], base=16), int(echonet_res[11][0], base=16))
      print('produce: %s, %s' % (key, value))
      produce_to_kafka(producer, key, value)

    sock.close()

データ送信のリクエストと受信は非同期なので、受信を待ち受けるスレッドを立ち上げる。

In [18]:
th = Thread(target=receive_state)
th.start()
time.sleep(3)

## リクエスト送信機能

別スレッドで結果を待ち受けているスレッドがある前提で、情報取得のリクエストを送る。

ここでも上記のECHONET Liteの通信プロトコルの定義をもとに、データ取得のためのリクエストを作成するヘルパー関数を定義する。

In [19]:
def create_command():
    # ---------------------------------------------------
    # 3.2.1 ECHONET Lite ヘッダ(EHD)
    # ---------------------------------------------------
    # 3.2.1.1 ECHONET Lite ヘッダ 1(EHD1)
    EHD1 = "10"  # ECHONET Lite規格
    
    # 3.2.1.2 ECHONET Lite ヘッダ 2(EHD2)
    EHD2 = "81"  # 形式１（規定電文形式）
    
    # 3.2.2 Transaction ID(TID)
    TID = "0001"  # IDなのでこの検証ではどの値でもOK
    
    # フレームのヘッダ－を構成
    EHD = EHD1 + EHD2 + TID
    
    # ---------------------------------------------------
    # 3.2.1 ECHONET Lite データ(EDATA)
    # ---------------------------------------------------
    # 3.2.4 ECHONETオブジェクト
    # EOJ = ECHONET Lite オブジェクト
    
    # SEOJ = 送信元ECHONET Lite オブジェクト
    SEOJ_CLS_GROUP = "05"     # 管理・操作関連クラスグループ
    SEOJ_CLS_CODE = "ff"      # コントローラー
    SEOJ_CLS_INSTANCE = "01"  # インスタンス番号
    SEOJ = SEOJ_CLS_GROUP + SEOJ_CLS_CODE + SEOJ_CLS_INSTANCE
    
    # DEOJ = 送信先ECHONET Lite オブジェクト
    DEOJ_CLS_GROUP = "01"     # 空調関連機器クラスグループ
    DEOJ_CLS_CODE = "30"      # 家庭用エアコンクラス
    DEOJ_CLS_INSTANCE = "01"  # All Instanses
    DEOJ = DEOJ_CLS_GROUP + DEOJ_CLS_CODE + DEOJ_CLS_INSTANCE
    
    
    # 3.2.5 ECHONET Lite サービス(ESV)
    ESV = "62"  # プロパティ値読み出し要求 (Get)
    
    ###############
    # APPENDIX ECHONET機器オブジェクト詳細規定
    # - 空調関連機器クラスグループ
    #  - 家庭用エアコンクラス規定
    # を確認
    
    # 3.2.6 処理プロパティカウンタ
    OPC = "02"  # 2件
    
    
    # プロパティ 1件目
    EPC1 = "80"    # 動作状態の取得(On or Off)
    PDC1 = "00"    # Getの場合は0でOK
    EDT1 = ""      # Getの場合はEDTは不要
    PROP1 = EPC1 + PDC1 + EDT1
    
    # プロパティ 2件目
    EPC2 = "b3"    # 温度設定の取得
    PDC2 = "00"    # Getの場合は0でOK
    EDT2 = ""      # Getの場合はEDTは不要
    PROP2 = EPC2 + PDC2 + EDT2
    
    
    # フレームのデータ部分であるEDATAを構成
    EDATA = SEOJ + DEOJ + ESV + OPC + PROP1 + PROP2
    
    echonet_command = EHD + EDATA

    return echonet_command

情報取得のためのリクエストを送信するヘルパー関数を定義する。

In [20]:
# `host` に 192.168.1.10を渡すとマルチキャストも利用できます
def send(host, echonet_command):
    echonet_port = EL_PORT
    
    # 要求送信用ソケットでコマンド送信
    send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    send_sock.sendto(bytes.fromhex(echonet_command), (host, echonet_port))
    send_sock.close()

今回は動作確認用の確認対象をあらかじめJupyterノートブックの起動時の環境変数に設定しておいた。
もし設定していない場合は、ここで任意のホストを指定するとよい。

In [21]:
target = os.environ['EL_TARGET']

ヘルパー関数を利用し、コマンドを作成する。

In [22]:
echonet_command = create_command()

試しに3回リクエストを送る。スリープ5秒を入れているので、その間にエアコンを操作すると、Kafkaに書き込まれる値が変わることがわかる。

In [23]:
for i in range(3):
    send(target, echonet_command)
    time.sleep(5)

# original
(b'\x10\x81\x00\x01\x010\x01\x05\xff\x01r\x02\x80\x011\xb3\x01\x14', ('10.0.4.73', 3610))
# parsed
[['0x10', '0x81'], ['0x0', '0x1'], ['0x1', '0x30', '0x1'], ['0x5', '0xff', '0x1'], ['0x72'], ['0x2'], ['0x80'], ['0x1'], ['0x31'], ['0xb3'], ['0x1'], ['0x14']]
produce: 0475#0x0,0x1, 49,20
Message produced: <cimpl.Message object at 0x7fdfde4fe5f0>
# original
(b'\x10\x81\x00\x01\x010\x01\x05\xff\x01r\x02\x80\x011\xb3\x01\x19', ('10.0.4.73', 3610))
# parsed
[['0x10', '0x81'], ['0x0', '0x1'], ['0x1', '0x30', '0x1'], ['0x5', '0xff', '0x1'], ['0x72'], ['0x2'], ['0x80'], ['0x1'], ['0x31'], ['0xb3'], ['0x1'], ['0x19']]
produce: 0475#0x0,0x1, 49,25
Message produced: <cimpl.Message object at 0x7fdfde4fe5f0>
# original
(b'\x10\x81\x00\x01\x010\x01\x05\xff\x01r\x02\x80\x010\xb3\x01\x19', ('10.0.4.73', 3610))
# parsed
[['0x10', '0x81'], ['0x0', '0x1'], ['0x1', '0x30', '0x1'], ['0x5', '0xff', '0x1'], ['0x72'], ['0x2'], ['0x80'], ['0x1'], ['0x30'], ['0xb3'], ['0x1'], ['0x19']]
produce: 0475#

ここでは `produce: 0475#0x0,0x1, 49,20` や `produce: 0475#0x0,0x1, 48,25` といった出力がみられているはずである。

これはApache Kafkaクラスタに書き込まれたデータを表しており、ダミーデータであるID（0475#0x0,0x1）、電源状態（48：On、49：Off）、温度設定値（上記例では20度、25度）という意味である。

# 読み込む

先ほど書き込んだメッセージを読みだして確認してみる。

動作確認のため、Apache Kafkaからの読み出し位置は最初から（OFFSET_BEGINNING）とする。

In [24]:
topic_partition = TopicPartition(topic_name, partition=0, offset=OFFSET_BEGINNING)
topic_partition

TopicPartition{topic=el_aircon,partition=0,offset=-2,error=None}

In [25]:
def my_on_assign(consumer, partitions):
    for p in partitions:
         # some starting offset, or use OFFSET_BEGINNING, et, al.
         # the default offset is STORED which means use committed offsets, and if
         # no committed offsets are available use auto.offset.reset config (default latest)
        p.offset = OFFSET_BEGINNING
    # call assign() to start fetching the given partitions.
    consumer.assign(partitions)

In [26]:
try:
    consumer = Consumer(consumer_conf)
    consumer.subscribe([topic_name], on_assign=my_on_assign)
    
    for i in range(10):
        msg = consumer.poll(timeout=1.0)
        if msg is None: continue
        
        if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
        else:
            print('%s : %s' % (msg.key(), msg.value()))
finally:
    consumer.close()

b'0748#0x0,0x1' : b'49,20'
b'0748#0x0,0x1' : b'49,20'
b'0748#0x0,0x1' : b'49,20'
b'0475#0x0,0x1' : b'49,20'
b'0475#0x0,0x1' : b'49,25'
b'0475#0x0,0x1' : b'48,25'


過去の動作確認で書き込んだ内容も含まれてしまっているかもしれないが、今回書き込んだデータも出力されたことがわかる。

# 後始末

必要に応じてトピックを削除する。
トピック削除しない場合はコメントアウトしておくこと。

In [27]:
#admin.delete_topics([topic_name])